## pull code

In [0]:
_git_branch = 'protocols-4'




import json
import subprocess
import sys
import time

import numpy as np
from IPython.core.display import display, HTML
from google.colab import files

!pip install overrides

Ничто = None



def exec(x):
  r = subprocess.check_output(x, shell=True)
  r = r.decode('unicode-escape').encode('latin1').decode('utf8')
  print(r)


print(f"fetching code from GitHub.....{_git_branch}")
try:
  exec('rm -r nlp_tools')
except:
  pass
exec(f'git clone --single-branch --branch {_git_branch} https://github.com/nemoware/analyser.git nlp_tools')

print('🦊 GIT revision:')
exec('cd nlp_tools\ngit rev-list --reverse HEAD | awk "{ print NR }" | tail -n 1\ngit branch\ngit log -3 --pretty=%B')

sys.path.insert(0, 'nlp_tools')

print('❤️importing Code from GitHub ... DONE')


#----
import matplotlib as mpl
from documents import TextMap
from colab_support.renderer import HtmlRenderer
from legal_docs import DocumentJson
 

class DemoRenderer(HtmlRenderer):
  def render_color_text(self, tokens, weights, colormap='coolwarm', print_debug=False, _range=None, separator=' '):
    html = self.to_color_text(tokens, weights, colormap, print_debug, _range, separator=separator)
    display(HTML(html))

  def to_color_text(self, tokens, weights, colormap='coolwarm', print_debug=False, _range=None, separator=' '):
    return super()._to_color_text(tokens, weights, mpl, colormap=colormap, _range=_range, separator=separator)

   
renderer_ = DemoRenderer()

def print_json_summary(cd:DocumentJson):
  wordsmap = TextMap(cd.normal_text, cd.tokenization_maps['$words'])
  print(f'read file {cd.filename}')

  for tag in cd.tags:
    span = tag.span
    _map = cd.tokenization_maps[tag.span_map]
    print(tag)
 
  

In [0]:
!pip install pyjarowinkler

### Init document-parser lib

In [0]:
lib_version = '1.1.9'
import os
if not os.path.isfile(f'document-parser-{lib_version}-distribution.zip'):
  !wget https://github.com/nemoware/document-parser/releases/download/$lib_version/document-parser-$lib_version-distribution.zip
if not os.path.isdir(f'document-parser-{lib_version}'):
  !unzip document-parser-$lib_version-distribution.zip

 
os.environ ['documentparser']=f'/content/document-parser-{lib_version}'
from integration.word_document_parser import WordDocParser, join_paragraphs
wp = WordDocParser()

### imports

In [0]:
import os
import pickle
import unittest

import numpy as np

from contract_parser import ContractAnlysingContext, ContractDocument
from contract_patterns import ContractPatternFactory
from legal_docs import LegalDocument
 
from ml_tools import *

# from headers_detector import doc_features, load_model, make_headline_attention_vector
from hyperparams import HyperParameters
from protocol_parser import protocol_votes_re
 

## 💅 Init Embedder(s)

In [0]:
from protocol_parser import ProtocolAnlysingContext, ProtocolPatternFactory
from tf_support.embedder_elmo import ElmoEmbedder
# from contract_patterns import ContractPatternFactory
elmo_embedder = ElmoEmbedder()
elmo_embedder_default = ElmoEmbedder(layer_name="default")

protocols_factory = ProtocolPatternFactory(elmo_embedder)
# contracts_factory = ContractPatternFactory(elmo_embedder)

In [0]:

#TODO: move to git hb
def calc_distances_to_pattern(sentences_embeddings_, pattern_embedding, dist_func=distance.cosine):
  assert len(pattern_embedding.shape) == 1
  assert len(sentences_embeddings_.shape) == 2
  assert sentences_embeddings_.shape[1] == pattern_embedding.shape[0] 

  _distances = np.ones(len(sentences_embeddings_))
  for word_index in range(0, len(sentences_embeddings_)):
    _distances[word_index] = max(0, min(1, 1.0 - dist_func(sentences_embeddings_[word_index], pattern_embedding)))

  return _distances

def calc_distances_per_pattern_dict( sentences_embeddings_, patterns_dict, patterns_embeddings):
  
  # TODO: see https://keras.io/layers/merge/#dot

  distances_per_pattern_dict = {}
  for i in range(len(patterns_dict)):
    pattern = patterns_dict[i]
    _distances = calc_distances_to_pattern(sentences_embeddings_, patterns_embeddings[i])

    distances_per_pattern_dict[pattern[0]] = _distances
  return distances_per_pattern_dict


## 👨🏿‍🚀 Test sentence embedding

In [0]:
# import tensorflow as tf
# import tensorflow_hub as hub
import matplotlib.pyplot as plt
# import numpy as np
# import os
# import pandas as pd
# import re
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity

def plot_similarity(labels, features, rotation):
#   corr = np.inner(features, features)
  corr = cosine_similarity(features, features)
  sns.set(font_scale=1.2)
  g = sns.heatmap(
      corr,
      xticklabels=labels,
      yticklabels=labels,
      vmin=0,
      vmax=1,
      annot=True, 
      cmap="GnBu" #"YlOrRd"
  )
  g.set_xticklabels(labels, rotation=rotation)
  g.set_title("Semantic Textual Similarity")


In [0]:
messages_ = ['мама раму мыла', 
             'мыла бодро раму мама', 
             'Антон хмурил брови', 
             'Антон скоро захватит мир', 
             'Антон не скоро захватит мир', 
             'Шла Саша мыть Раму',
             'Шла Саша мыть раму довольно бодро']
message_embeddings_ = elmo_embedder_default.embedd_strings(messages_)
plot_similarity(messages_, message_embeddings_, 90)

## Embedd patterns as sentences


### print patterns

In [0]:
patterns = [ ' '.join(p.prefix_pattern_suffix_tuple) for p in protocols_factory.patterns]
patterns

In [0]:
for p in  protocols_factory.patterns:
  pn = '===='.join(p.prefix_pattern_suffix_tuple)
  print(p.name, p.prefix_pattern_suffix_tuple[1], pn)
  print('--')

### Read doc

In [0]:
#@title ✅Pre-embedded or Upload?
upload_files = True #@param {type:"boolean"}
filename = '/content/\u041F\u0440\u043E\u0442\u043E\u043A\u043E\u043B_\u0413\u041F\u041D-\u041A\u041F_\u0417\u0435\u043B\u0435\u043D\u044B\u0435.doc' #@param {type:"string"}
if not upload_files:
  with open('/content/nlp_tools/tests/Протокол_СД_ 3.docx.pickle', 'rb') as handle:
    doc = pickle.load(handle)

if upload_files:
  results = wp.read_doc(filename)
  for doc in results['documents'][:1]:  # XXX
    if 'PROTOCOL' == doc['documentType']:    
      doc = join_paragraphs(doc, 'no_id')


from legal_docs import tokenize_doc_into_sentences_map
sentence_map = tokenize_doc_into_sentences_map(doc, 250)
 
for token in sentence_map.tokens[0:30]:
  print('-->', token)

## define patterns dict

In [0]:
patterns_dict=[   
    ['sum_max1', 'стоимость не более 0 млн. тыс. миллионов тысяч рублей долларов копеек евро'],

    # ['solution_1','решение, принятое по вопросу повестки дня:'],
    # ['solution_2','по вопросам повестки дня приняты следующие решения:'],
    
    ['not_value_1', 'размер уставного капитала 0 рублей'],
    ['not_value_2', 'принятие решения о назначении секретаря'],
   
    ['agenda_end_1','кворум для проведения заседания и принятия решений имеется'],
    ['agenda_end_2','Вопрос повестки дня заседания'],
    ['agenda_end_3','Формулировка решения по вопросу повестки дня заседания:'],
    
    ['agenda_start_1','повестка дня заседания'],
    ['agenda_start_2','Повестка дня'],
    
    ['deal_approval_1', 'одобрить совершение сделки'],
    ['deal_approval_1.1', 'одобрить сделку'],
    ['deal_approval_2','дать согласие на заключение договора'],
    ['deal_approval_3','принять решение о совершении сделки'],
    ['deal_approval_3.1','принять решение о совершении крупной сделки'],
    ['deal_approval_4','заключить договор аренды'],
    
    ['question_1','По вопросу № 0'],
    ['question_2', 'Первый вопрос повестки дня заседания'],
    ['question_3', 'Решение, принятое по вопросу повестки дня:'],
    ['question_4', 'Решение, принятое по 1 вопросу повестки дня:'],

    ['footers_1', 'Время подведения итогов голосования'],
    ['footers_2', 'Список приложений:'],
    ['footers_3', 'Подсчет голосов производил Секретарь Совета директоров'],
    ['footers_4', 'Протокол составлен в 2-х экземплярах']
    
    
    
]

patterns_te=[ p[1] for p in patterns_dict ]
patterns_embeddings_ = elmo_embedder_default.embedd_strings(patterns_te)

### ⚙️🔮SENTENCES embedding

In [0]:
sentences_embeddings_ = elmo_embedder_default.embedd_strings(sentence_map.tokens)


### ⚙️🔮 WORDS Ebmedding 

In [0]:
protocols_factory = ProtocolPatternFactory(elmo_embedder)
doc.embedd(protocols_factory)
doc.calculate_distances_per_pattern(protocols_factory)
print()

### 👨🏿‍🚀Try find subject

In [0]:
subject = '''продажей недвижимого имущества, принадлежащего Обществу на праве собственности'''

# subject = '''договор аренды нежилого помещения'''

subject = subject[:200]
subject_embeddings = elmo_embedder_default.embedd_strings([subject])

import scipy.spatial.distance as distance



contract_subj_attention = calc_distances_to_pattern(sentences_embeddings_, subject_embeddings[0])
contract_subj_attention = relu(contract_subj_attention, 0.5)



renderer_.render_color_text(sentence_map.tokens, contract_subj_attention, _range=(0,1), separator='<br>')



## Split text by agenda questions



In [0]:
def collect_spans_having_votes(segments, textmap):
  """
  search for votes in each document segment
  collect only
  :param segments: 
  :param textmap: 
  :return:  segments with votes
  """
  for span in segments:
    # print('=' * 50)
    subdoc = textmap.slice(span)
    protocol_votes = list(subdoc.finditer(protocol_votes_re))
    if protocol_votes:
      # print('-' * 50)
      # print(subdoc.text)
      # for c in protocol_votes:
      #   print('found:', subdoc.text_range(c))
      yield span
    # else:
    #   print('not found')
    

### 📝 Sections attention

In [0]:
def find_protocol_sections_edges(distances_per_pattern_dict):

  patterns = ['deal_approval_', 'footers_', 'question_']
  vv_=[]
  for p in patterns:
    v_ = max_exclusive_pattern_by_prefix(distances_per_pattern_dict, p)
    v_ = relu(v_, 0.5)
    vv_.append(v_)
  
  v_sections_attention = sum_probabilities(vv_)

  # v_deal_approval = max_exclusive_pattern_by_prefix(distances_per_pattern_dict, 'deal_approval_')
  # v_deal_approval = relu(v_deal_approval, 0.5)

  # v_question = max_exclusive_pattern_by_prefix(distances_per_pattern_dict, 'question_')
  # v_question = relu(v_question, 0.5)

  # v_sections_attention = sum_probabilities([v_deal_approval, v_question])
  v_sections_attention = relu(v_sections_attention, 0.7)
  return v_sections_attention


distances_per_pattern_dict = calc_distances_per_pattern_dict(sentences_embeddings_, patterns_dict, patterns_embeddings_)
v_sections_attention = find_protocol_sections_edges(distances_per_pattern_dict)

# renderer_.render_color_text(sentence_map.tokens , v_question+v_deal_approval, _range=(0,1), separator='<br>')
renderer_.render_color_text(sentence_map.tokens,  v_sections_attention, _range=(0,1), separator='<br>')



In [0]:
question_spans_sent = spans_between_non_zero_attention(v_sections_attention)
print('question_spans_sent', question_spans_sent)

question_spans_words =  sentence_map.remap_slices(question_spans_sent,  doc.tokens_map )
print('question_spans_words', question_spans_words)

More attention to spans having votes

In [0]:
spans_having_votes = list(collect_spans_having_votes(question_spans_sent, sentence_map))


spans_having_votes_words =  sentence_map.remap_slices(spans_having_votes,  doc.tokens_map )
# questions_attention =  spans_to_attention(question_spans_words, len(doc))
votes_attention =  spans_to_attention(spans_having_votes_words, len(doc))

# v_deal_approval_words = sentence_map.remap_spans(v_deal_approval,  doc.tokens_map )
v_deal_approval = max_exclusive_pattern_by_prefix(distances_per_pattern_dict, 'deal_approval_')
_spans, v_deal_approval_words_attention =  sentences_attention_to_words(v_deal_approval, sentence_map,  doc.tokens_map)




### debug -------------votes_attention

renderer_.render_color_text(doc.tokens_map.tokens , relu(v_deal_approval_words_attention, 0.3)   , _range=(0,1))

for span in spans_having_votes:
  print(span, '=' * 50)
  subdoc = sentence_map.slice(span)
  print(subdoc.text)
  print(sentence_map.char_range( [span.start, span.stop] ))
  print('^' * 50)
  


### Value attention

In [0]:

s_value_attention_vector = max_exclusive_pattern_by_prefix(doc.distances_per_pattern_dict, 'sum_max_p_')
s_value_attention_vector_neg = max_exclusive_pattern_by_prefix(doc.distances_per_pattern_dict, 'sum_max_neg')
s_value_attention_vector -=   s_value_attention_vector_neg/3
s_value_attention_vector = relu(s_value_attention_vector, 0.25)

v_deal_approval_words_attention = relu (v_deal_approval_words_attention, 0.5)
value_attention_vector = sum_probabilities ( [s_value_attention_vector, v_deal_approval_words_attention, votes_attention/3.0] )
# value_attention_vector = sum_probabilities ( [ v_deal_approval_words_attention, votes_attention/10.0] )
value_attention_vector = relu(value_attention_vector, 0.5)
 
renderer_.render_color_text(doc.tokens_map.tokens, value_attention_vector  )

In [0]:
#sentences to words
from contract_parser import find_value_sign_currency_attention 
from legal_docs import ContractValue

words_spans_having_votes =  sentence_map.remap_slices (spans_having_votes,  doc.tokens_map )


#--------------
#color things
print('spans_having_votes', list(spans_having_votes))
print('words_spans_having_votes', list(words_spans_having_votes))

#

# for span in list(words_spans_having_votes):
#   print(span)
#   selection [ span[0]:span[1]+1  ] += 0.4
#--------------

#find contract values
# for span in words_spans_having_votes:
#   subdoc = doc.subdoc_slice(slice(span[0],span[1]) )
values:List[ContractValue] = find_value_sign_currency_attention(doc, value_attention_vector)

#color 
numbers_attention = np.zeros(len(doc.tokens_map))
numbers_confidence = np.zeros(len(doc.tokens_map))
for v in values:
  numbers_confidence [ v.value.as_slice() ] +=v.value.confidence
  numbers_attention  [ v.value.as_slice() ] = 1
  numbers_attention  [ v.currency.as_slice() ] = 1
  numbers_attention  [ v.sign.as_slice() ] = 1

block_confidence = sum_probabilities ( [numbers_attention, v_deal_approval_words_attention, votes_attention/5] )

# value_attention_vector = max_exclusive_pattern_by_prefix(doc.distances_per_pattern_dict, '_phrase')
renderer_.render_color_text(doc.tokens_map.tokens, numbers_attention, _range=(0,1))


## Collect only meaningful sections

In [0]:
# block_confidence = sum_probabilities ( [ numbers_attention, value_attention_vector, v_deal_approval_words_attention] )

renderer_.render_color_text(doc.tokens_map.tokens, block_confidence, _range=(0,1))


In [0]:
def find_confident_spans(slices, block_confidence, tag_name):
  k = 0
  for _slice in slices:
    k+=1
    pv = block_confidence[_slice[0]:_slice[1]]
    confidence = estimate_confidence_by_mean_top_non_zeros(pv, 5)
    print('-' * 100)
    print(_slice, confidence)
    if confidence > 0.6:
      st = SemanticTag(f"{tag_name}_{k}", None, _slice)
      st.confidence = confidence
      yield (st)


tags = find_confident_spans(question_spans_words, block_confidence, 'agenda_item' )
for tag in tags:
  print(tag.kind)
  print(doc.substr(tag))

# MISCL

In [0]:
def detect_agenda(distances_per_pattern_dict):
  v1 = max_exclusive_pattern_by_prefix(distances_per_pattern_dict,  'agenda_start_' )
  agenda_index_start = np.argmax(v1)
  # v =max_exclusive_pattern_by_prefix( [distances_per_pattern_dict['deal_approval_1'], distances_per_pattern_dict['deal_approval_2']]  )
  v2 = max_exclusive_pattern_by_prefix(distances_per_pattern_dict,  'agenda_end_' )
  agenda_index_end = np.argmax(v2)

  v2[:agenda_index_start] = 0
  v2[agenda_index_end:] = 0


  return None, v2
  pass

questions, attention_v = detect_agenda(distances_per_pattern_dict)
print(questions)
renderer_.render_color_text(spans_map.tokens , attention_v, _range=(0,1), separator='<br>')
# plot_similarity(messages_, message_embeddings_, 90)

In [0]:
doc[0:4].text

In [0]:

from contract_agents import find_org_names
from typing import List

def find_protocol_org(protocol)->List[SemanticTag]:
  ret=[]
  x:List[SemanticTag] = find_org_names(protocol[0:200])
  ret.append( SemanticTag.find_by_kind(x, 'org.1.name'))
  ret.append( SemanticTag.find_by_kind(x, 'org.1.type'))
  return x

x=find_protocol_org(doc)
for c in x:
  print(c)

## topic_attention

In [0]:
topic_attention = make_topic_attention(doc, headline_attention_vector, 'p_agenda_')
topic_attention = relu(topic_attention, 0.5)
# renderer_.render_color_text(doc.tokens_cc, topic_attention)

def mark_section(topic_attention:FixedVector) -> FixedVector:
  section_paint = np.zeros_like(topic_attention)
  start = np.argmax(topic_attention)
  start_v=topic_attention[start]
  section_paint[start:] = start_v

  return section_paint


  print(start)

_sp = mark_section(topic_attention)
renderer_.render_color_text(doc.tokens_cc, _sp, _range=(0,1))

In [0]:

all_headlines_indices = all_headlines_spans[:,0]

# ---
def find_sections_spans_by_indices_2(_doc: LegalDocument,
                                   headline_pattern_names,
                                   all_headlines_indices,
                                   headlines_mask,
                                   topic,
                                   relu_threshold=0.6):
  av_name = f'{topic}_headlines_attention_vector'
  print("TOPIC:", av_name)
  pv = _doc.distances_per_pattern_dict
  pv[av_name] = relu(combined_attention_vectors(pv, headline_pattern_names), relu_threshold)

  # masking
  pv[av_name] *= headlines_mask

  _headlines_of_interest_spans = find_sentences_with_attention(_doc.tokens_map, pv[av_name])
  _headlines_of_interest_indices = _headlines_of_interest_spans[:, 0]

  for i in _headlines_of_interest_indices:
    last = find_first_gt(i, all_headlines_indices)
    if last is None:
      last = len(_doc.tokens_map) - 1
    yield [i, last]
  pass


##==========
solution_spans = find_sections_spans_by_indices_2 (
    doc, solution_pattern_names, 
    all_headlines_indices,
    headline_attention_vector_relu, 
    topic='solutions', 
    relu_threshold=0.6)

agenda_spans   = find_sections_spans_by_indices_2 (
    doc, agenda_pattern_names, 
    all_headlines_indices, 
    headline_attention_vector_relu, 
    topic='agenda', 
    relu_threshold=0.6)

_agenda_hd_v = doc.distances_per_pattern_dict['agenda_headlines_attention_vector']
_solutions_hd_v = doc.distances_per_pattern_dict['solutions_headlines_attention_vector']

for agenda_span in agenda_spans:
  print('agenda_span', agenda_span)
  # print(doc.tokens_map.text_range(agenda_span))
  _slice = slice(agenda_span[0], agenda_span[1])
  renderer_.render_color_text(doc.tokens_map [_slice ], _filtered_headline_attention_vector[_slice])

print('*'*100)

print('='*100)
for solution_span in solution_spans:
  print(':::::: solution_span', solution_span)
  print('- '*50)
  # print('☢️ ', doc.tokens_map.text_range(solution_span))
  _slice = slice(solution_span[0],solution_span[1])
  renderer_.render_color_text(doc.tokens_map [_slice ], _filtered_headline_attention_vector[_slice])
  print('-'*100)

 

In [0]:
list(doc.distances_per_pattern_dict.keys())

In [0]:
renderer_.render_color_text(doc.tokens_cc, _solutions_hd_v)

# Test on uploaded file

In [0]:
# del doc

In [0]:
from integration.word_document_parser import WordDocParser, join_paragraphs

document_parser_v='1.0.9'

!wget https://github.com/nemoware/document-parser/releases/download/$document_parser_v/document-parser-$document_parser_v-distribution.zip
!unzip document-parser-$document_parser_v-distribution.zip  
os.environ ['documentparser']=f'/content/document-parser-{document_parser_v}'

wordDocParser = WordDocParser()

## read file

In [0]:
json_res = wordDocParser.read_doc('/content/Протокол_ООО Вектор продажа недвиж.doc')


In [0]:
from headers_detector import make_headline_attention_vector
from integration.word_document_parser import join_paragraphs
from ml_tools import find_non_zero_spans, merge_colliding_spans, relu


def analyse_protocol(json_res, ctx):
  #embedding, etc
  protocol = join_paragraphs(json_res, '0')
  protocol.embedd_tokens(ctx.protocols_factory.embedder)
  #TODO: tetect agents
  vd = protocol.calculate_distances_per_pattern(ctx.protocols_factory, verbosity=0)
  
  vd['headline_attention_vector'] = make_headline_attention_vector(protocol)
  vd['headline_attention_vector_relu'] = relu(vd['headline_attention_vector'], HyperParameters.parser_headline_attention_vector_denominator)

  all_headlines_spans = find_sentences_with_attention(protocol.tokens_map, vd['headline_attention_vector_relu'])
  
  filtered_all_headlines_spans = merge_colliding_spans(all_headlines_spans)
  all_headlines_indices = filtered_all_headlines_spans[:,0]

  # test
  vd['filtered_headline_attention_vector'] = np.zeros_like(vd['headline_attention_vector_relu'])
  for s in filtered_all_headlines_spans:
    vd['filtered_headline_attention_vector'][s[0]:s[1]] += 1

  return protocol, all_headlines_indices


protocol, all_headlines_indices = analyse_protocol(json_res, ctx)




## Solutions (решения по вопросам)

In [0]:

solution_spans = find_sections_spans_by_indices (protocol, 
                                                 solution_pattern_names, 
                                                 all_headlines_indices,
                                                 protocol.distances_per_pattern_dict['headline_attention_vector_relu'], 
                                                 'solution',
                                                  relu_threshold=0.75)


print('='*100)
for solution_span in solution_spans:
  print(':::::: solution_span', solution_span)
  print('- '*50)
  print('☢️ ', protocol.tokens_map.text_range(solution_span))
  print('-'*100)



## headline_attention_vector 


In [0]:
renderer_.render_color_text(protocol.tokens_cc, protocol.distances_per_pattern_dict['headline_attention_vector'])

## headline_attention_vector_relu

In [0]:
renderer_.render_color_text(protocol.tokens_cc, protocol.distances_per_pattern_dict['headline_attention_vector_relu'])

## filtered_headline_attention_vector

# Latent mean (center) sense

In [0]:
TextDocument = LegalDocument #alias

In [0]:
from patterns import *

def latent_sense(embeddings, wnd_half):
   
  R = np.zeros(len(embeddings))
  for i in range (len(embeddings)):
    _start = 0
    if i > wnd_half: _start = i-wnd_half
    word_emb = embeddings[i:i+1]
    window_emb = embeddings[_start : i+wnd_half]   
    semantic_similarity =  1- dist_mean_cosine(window_emb, word_emb)
    R[i] = semantic_similarity
  
  return R

def render_any(t, v):
  renderer_.render_color_text(t, v, _range=(min(v), max(v)) )


In [0]:
# text = """
# В соответствии с пп . 25 пункт 10.3. статьи 10 Устава ООО « Технологический центр « Бажен » принято следующее решение :
# на основании статьи 17.1 . Закона РФ от 21-02-1992 № 2395-1 « О недрах » ( юридическое лицо - пользователь недр выступает учредителем нового юридического лица , созданного для продолжения деятельности на предоставленном участке недр в соответствии с лицензией на пользование участком недр , при условии , если новое юридическое лицо образовано в соответствии с законодательством Российской Федерации и ему передано имущество , необходимое для осуществления деятельности , указанной в лицензии на пользование участком недр , в том числе из состава имущества объектов обустройства в границах участка недр , а также имеются необходимые разрешения ( лицензии ) на осуществление видов деятельности , связанных с недропользованием , и доля прежнего юридического лица - пользователя недр в уставном капитале нового юридического лица на момент перехода права пользования участком недр составляет не менее половины уставного капитала нового юридического лица ) одобрить передачу ООО « Технологический центр « Бажен » Обществу с ограниченной ответственностью « Газпромнефть - Салым » , созданному на основании решения учредителей , права пользования недрами в пределах :
# 1. участка Салымский 3 , расположенного в Ханты-Мансийском районе Ханты-Мансийского автономного округа – Югры Тюменской области ( лицензия на пользование недрами ХМН 03590 НР , зарегистрирована 10-04-2019 года , с целевым назначением и видами работ – для геологического изучения , включающего поиски и оценку месторождений полезных ископаемых , разведки и добычи полезных ископаемых ) ;
# 2. участка Салымский 5 , расположенного в Ханты-Мансийском районе Ханты-Мансийского автономного округа – Югры Тюменской области ( лицензия на пользование недрами ХМН 03590 НР зарегистрирована 10-04-2019 года , с целевым назначением и видами работ – для геологического изучения , включающего поиски и оценку месторождений полезных ископаемых , разведки и добычи полезных ископаемых ) ;
# 3. участка Салымский 3 , расположенного в Ханты-Мансийском районе Ханты-Мансийского автономного округа – Югры Тюменской области ( лицензия на пользование недрами ХМН 20562 ВЭ , зарегистрирована 21-06-2019 года , с целевым назначением и видами работ – добыча подземных вод для технологического обеспечения водой объектов промышленности ) .
# Право пользования участками недр передается путем переоформления лицензий на право пользования недрами , выданных ООО « Технологический центр « Бажен » , в порядке , установленном Законом РФ от 21-02-1992 № 2395-1 « О недрах » .
# """
 
td = doc
td.parse()
td.embedd_tokens(elmo_embedder)
td=doc


local_sense = latent_sense(td.embeddings,  7 ) 
# mean_sense -= min(mean_sense)


# R
# mean_sense = relu(R, np.mean(R)*1.1)

render_any(td.tokens_cc,  local_sense )
render_any(td.tokens_cc,  local_sense-mean_sense)


# meaninful_tokens = np.nonzero(R)[0:][0]
# print('-'*100)
# print(f'tokens: {len(td.tokens)} \t meaninful {len(meaninful_tokens)}')
# print('-'*100)
# res=[ td.tokens_map.text_range([i, i+1]) for i in meaninful_tokens ]
# print(' ' .join(res) )

### Latent mean (mean-ing)

In [0]:
mean_sense = latent_sense(td.embeddings,  120 )

meaningless = relu (mean_sense, (np.max(mean_sense) + np.mean(mean_sense))/2  )
outliers = relu (1.0 - mean_sense,0.8)

render_any(td.tokens_cc, meaningless)

In [0]:
# 


### Latent local mean (mean-ing) without nonsense

In [0]:
mean_local_sense = latent_sense(td.embeddings,  7 )
nonsense_subtract = mean_local_sense - meaningless
nonsense_subtract = relu(nonsense_subtract, 0.2)
render_any(td.tokens_cc, nonsense_subtract )

# Pattern search (distance functions)

In [0]:
test_pattern = ctx.protocols_factory.create_pattern('test', ('','Принятие решения о заключении ООО « Технологический центр « Бажен » договора аренды недвижимого имущества с Обществом с ограниченной ответственностью « Ренова Лаб » ;',''))
test_pattern2 = ctx.protocols_factory.create_pattern('test2', ('','По вопросу № 7',''))
ctx.protocols_factory.embedd()

In [0]:
# def per_token_dist_mean_cosine(window, pattern):
   
#   d=0
  
#   for p in pattern:
#     for v in window:
#       d += distance.cosine(p, v)
  
#   return d / (len(window) * len(pattern))
def dist_cosine_to_point( text_emb, pt ):
  t_distances = np.zeros(len(text_emb))
  for i in range(len(text_emb)):
    # print(text_emb[i][0].shape, pt[0].shape)
    d = distance.cosine (text_emb[i], pt) 
    if d>1: d=1    
    t_distances[i] = d

  # print(min(t_distances), max(t_distances))
  
  return relu(t_distances,0.5)

def per_token_similarity_cosine( text_emb, pattern_emb ):   
  a_distances = np.zeros(len(text_emb))
  for p in pattern_emb:
    t_distances = 1 - dist_cosine_to_point(text_emb, p)    
    # a_distances = sum_probabilities ([ t_distances, a_distances ])
    a_distances = max_exclusive_pattern ([ t_distances, a_distances ])
    
  return a_distances

test_dists = per_token_similarity_cosine(doc.embeddings, test_pattern.embeddings )
# test_dists = relu (1-test_pattern._eval_distances(doc.embeddings), 0.5)
test_dists = relu(test_dists, 0.5)
render_any(td.tokens_cc, test_dists)



# distance.cosine (test_pattern.embeddings[1], doc.embeddings[1]) 